# Particles

In [ ]:
import numpy as np
import osyris

In [ ]:
%matplotlib widget

In [ ]:
path = "osyrisdata/cosmology"
data = osyris.Dataset(100, scale="Mpc", path=path).load('part')

In [ ]:
osyris.histogram2d(data["part"]["position"].x,
                   data["part"]["position"].y,
                   resolution=512, norm="log")

In [ ]:
import pythreejs as p3
geometry = p3.BufferGeometry(
    attributes={
        'position': p3.BufferAttribute(array=data["part"]["position"].values / 10. - 7.5)
    })
material = p3.PointsMaterial(color='black', size=0.1)
points = p3.Points(geometry=geometry, material=material)
view_width = 800
view_height = 500
camera = p3.PerspectiveCamera(position=[20.0, 0, 0], aspect=view_width/view_height)
scene = p3.Scene(children=[points, camera], background="#DDDDDD")
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderer